In [7]:
import ipywidgets as widgets
from IPython.display import display



In [2]:
#common widgets layout

label_layout = widgets.Layout(display="flex", justify_content="flex-end", width="90%")

box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center')

sidebar_layout = widgets.Layout(width='98%')

desc_width = {'description_width': '120px'}

In [3]:
satellite_list = [
    'Landsat 7 SR',

    'Landsat 8 SR',
    'Landsat 9 SR',

    'Landsat 7 TOA',

    'Landsat 8 TOA',
    'Landsat 9 TOA',
    'Sentinel-2 SR',
    'Sentinel-2 TOA']

collections_list = [
    'LANDSAT/LE07/C02/T1_L2',

    'LANDSAT/LC08/C02/T1_L2',
    'LANDSAT/LC09/C02/T1_L2',

    'LANDSAT/LE07/C02/T1_TOA',
    'LANDSAT/LC08/C02/T1_TOA',
    'LANDSAT/LC09/C02/T1_TOA',
    'COPERNICUS/S2_SR_HARMONIZED',
    'COPERNICUS/S2_HARMONIZED']

bands_list = [
    ['SR_B'+str(i) for i in range(1,6)] + ['SR_B7', 'ST_B6'],
    ['SR_B'+str(i) for i in range(1,8)] + ['ST_B10'],
    ['SR_B'+str(i) for i in range(1,8)] + ['ST_B10'],

    ['B'+str(i) for i in range(1,6)] + ['B7', 'B8'],
    ['B'+str(i) for i in range(1,12)],
    ['B'+str(i) for i in range(1,12)],
    ['B'+str(i) for i in range(1,10)] + ['B8A', 'B11', 'B12'],
    ['B'+str(i) for i in range(1,13)] + ['B8A'],
]

w_satellite = widgets.Dropdown(
    options=satellite_list,
    value='Landsat 9 SR',
    description='Satellite',
    disabled=False,
    style=desc_width,
    layout=sidebar_layout
)

ind = satellite_list.index(w_satellite.value)

w_select_bands = widgets.SelectMultiple(
    options=bands_list[ind],
    value=bands_list[ind][1:6],
    rows=6,
    description='Bands',
    style=desc_width,
    layout=sidebar_layout,
    disabled=False
)

def satellite_handler(change):
  ind = satellite_list.index(change.new)
  #print(change.new, ', bands[{}]'.format(ind), bands_list[ind])
  w_select_bands.options = bands_list[ind]
  w_select_bands.value = bands_list[ind][1:6]

w_satellite.observe(satellite_handler, names='value')

#display(w_satellite, w_select_bands)


In [5]:
display(w_satellite, w_select_bands)

Dropdown(description='Satellite', index=2, layout=Layout(width='98%'), options=('Landsat 7 SR', 'Landsat 8 SR'…

SelectMultiple(description='Bands', index=(1, 2, 3, 4, 5), layout=Layout(width='98%'), options=('SR_B1', 'SR_B…

### Region of interest

There are several ways you can create a region for generating the training dataset.

- Draw a shape (e.g., rectangle) on the map and then use `region = Map.user_roi`
- Define a geometry, such as `region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])`
- Create a buffer zone around a point, such as `region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)`
- If you don't define a region, it will use the image footprint by default

```
image = (
  ee.ImageCollection(collection)
    .filterBounds(region)
    .filterDate(start_date, end_date)
    .map(maskL8sr)
    .median()
    .select(bands)
    .clip(region)
)
```

In [8]:
region_options = [
    'Select an option', # if unselected, default to map bounds
    'Draw shapes on map', # rectangle, polygon. If a point, use next option
    'Input point and buffer', # input point coordinates and buffer distance
    'Rectangle from BBox', # input bounding box coordinates
    'Upload GeoJSON', # upload geometry in .GeoJSON
    ]

w_region = widgets.Dropdown(
    options=region_options,
    value='Select an option',
    description='Region',
    disabled=False,
    style=desc_width,
    layout=sidebar_layout
)

w_region_detail = widgets.VBox()

def regions_handler(change):
  if change.new == 'Draw shapes on map':
    w_region_detail.children = [widgets.Label(value="Please draw geometry on the map.",
                                              layout=label_layout)]

  elif change.new == 'Input point and buffer':
    w_point = widgets.Text(description='Lon, Lat',
                           style=desc_width,
                           layout=sidebar_layout)
    w_buffer = widgets.FloatText(description='Buffer (km)',
                                 style=desc_width,
                                 layout=sidebar_layout)
    w_region_detail.children = [w_point, w_buffer]

  elif change.new == 'Rectangle from BBox':
    w_points = widgets.Text(description='BBox (W, S, E, N)',
                            style=desc_width,
                            layout=sidebar_layout)
    w_region_detail.children = [w_points]

  elif change.new == 'Upload GeoJSON':
    w_btn_region = widgets.Button(description='Upload GeoJSON',
                            button_style='danger',
                            tooltip="Upload GeoJSON",
                            #style={'font_size': '3px', 'font_weight': 'bold'}
                            )

    w_btn_region_output = widgets.Output()
    def btn_region_handler(obj):
        w_btn_region_output.clear_output()
        with w_btn_region_output:
          uploaded = files.upload()
          for fn in uploaded.keys():
            global region_json
            region_json = fn
            #print(region_json)

    w_btn_region.on_click(btn_region_handler)
    w_region_detail.children = [w_btn_region, w_btn_region_output]
    #w_region_detail.layout = box_layout

  else:
    w_region_detail.children = [widgets.Label(
        value="Default to current map boundary.",
        layout=label_layout)]

w_region.observe(regions_handler, names='value')
w_regions = widgets.VBox([w_region, w_region_detail])
#display(w_regions)

In [9]:
display(w_regions)

### DatePicker & temporal interval

In [10]:
# could use string as well
w_start_date = widgets.DatePicker(
    description='Start date',
    disabled=False,
    style=desc_width,
    layout=sidebar_layout
)

w_end_date = widgets.DatePicker(
    description='End date',
    disabled=False,
    style=desc_width,
    layout=sidebar_layout
)

w_separate_doy = widgets.Checkbox(description='Separate doy from year',
                                  value=False,
                                  style=desc_width,
                                  layout=sidebar_layout)

w_temporal = widgets.Checkbox(description='Temporal composite',
                              value=False,
                              style=desc_width,
                              layout=sidebar_layout)

temporal_options = [
    'All dates', # just median temporal composite of all scenes
    'Yearly', # temporal composite for each year of all scenes
    'Monthly', # temporal composite for each month of all scenes
    'Daily', # temporal composite for each day of all scenes
]

w_temporal = widgets.Dropdown(
    options=temporal_options,
    value='All dates',
    description='Composite interval',
    disabled=False,
    style=desc_width,
    layout=sidebar_layout
)

w_temporal_detail = widgets.VBox()

def temporal_handler(change):
  if change.new == 'All dates':
    w_temporal_detail.children = [widgets.Label(value="Median composite for the entire interval",
                                                layout=label_layout)]
  elif change.new == 'Yearly':
    w_temporal_detail.children = [widgets.Label(value="Yearly median composite",
                                                layout=label_layout)]
  elif change.new == 'Monthly':
    w_temporal_detail.children = [widgets.Label(value="Monthly median composite",
                                                layout=label_layout)]
  elif change.new == 'Daily':
    w_temporal_detail.children = [widgets.Label(value="Daily mosaic",
                                                layout=label_layout)]

w_temporal.observe(temporal_handler, names='value')

w_dates = widgets.VBox([
    w_start_date,
    w_end_date,
    #w_separate_doy,
    w_temporal,
    w_temporal_detail
])


In [11]:
display(w_dates)

In [12]:
import geemap
Map = geemap.Map()


Successfully saved authorization token.


In [14]:
w_btn_workflow_log = widgets.Output(layout={'border': '2px solid blue',
                                            'height': '200px',
                                            'overflow': 'auto'})

def runApp():
  """
  Main app function
  """

  header = widgets.HTML("<h1 style='text-align: center'>An Explainable ML Land Cover Mapping with GEE</h1>")

  left_sidebar=widgets.VBox([
        w_regions,
        w_dates,
        w_satellite,
        w_select_bands
        #w_btn_features,
        ], layout=widgets.Layout(width='28%')
  )

  app = widgets.VBox([
      header,
      widgets.HBox([left_sidebar,
                    widgets.VBox([Map, w_btn_workflow_log],
                                 layout=widgets.Layout(width='72%'))
                  ]),

      #bottom_tab,
  ])

  display(app)

runApp()